# Setup Dataset

In [27]:
import torch 
import matplotlib.pyplot as plt
from torchvision.models import resnet50, resnet152, resnet18
from tqdm import tqdm
import numpy as np
from torchvision.models.densenet import DenseNet
from torchsummary import summary
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets.folder import DatasetFolder
import torchvision
import os
import gc
import time
import copy
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from collections import defaultdict, Counter

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as pt_optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.optim.lr_scheduler as pt_lr_scheduler
from torchvision.datasets.folder import DatasetFolder

In [24]:
import os

try:
  import google.colab
  COLAB_ENV = True
except:
  COLAB_ENV = False


if COLAB_ENV:
  data_filepath = "/" + os.path.join("content","drive","MyDrive","datasets", "ML4SCI_GSOC23")
  SAVE_MODEL_PATH = os.path.join(data_filepath, "gravitational-lensing", "Task1", "models")
  os.makedirs(SAVE_MODEL_PATH, exist_ok=True)
  os.listdir(SAVE_MODEL_PATH)
  from google.colab import drive
  drive.mount('/content/drive')
else:
  data_filepath = "data"
     

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp /content/drive/MyDrive/datasets/ML4SCI_GSOC23/gravitational-lensing/Task1/dataset.zip .
!unzip -q dataset.zip

In [25]:
import os

def getFilenamesFullPath(path):
  no_path = os.path.join(*path)
  no = os.listdir(no_path)
  return sorted([os.path.join(no_path, v) for v in no])

dataset_filepath_train = os.path.join("dataset", "train")
dataset_filepath_val = os.path.join("dataset", "val")

print(os.listdir(dataset_filepath_train))

no = getFilenamesFullPath(["dataset", "train", "no"])
vort = getFilenamesFullPath(["dataset", "train", "vort"])
sphere = getFilenamesFullPath(["dataset", "train", "sphere"])

print(f"size of no class : {len(no)}")
print(f"size of vort class : {len(vort)}")
print(f"size of sphere class : {len(sphere)}")

['no', 'vort', 'sphere', '.DS_Store']
size of no class : 10000
size of vort class : 10000
size of sphere class : 10000


In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device Used : {device}")

mean = 0.485
std = 0.229

mean = 0.06169275318411914
std = 0.1172684106873827
batch_size = 64

_transforms = transforms.Compose([
    torchvision.transforms.Normalize((mean), (std))
])

def load_npy(path : str) -> torch.Tensor:
    # load the image
    img = torch.from_numpy(np.load(path))

    # preprocess it     
    return _transforms(img)

train_ds = DatasetFolder(dataset_filepath_train, extensions=[".npy"], loader=load_npy)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

test_ds = DatasetFolder(dataset_filepath_val, extensions=[".npy"], loader=load_npy)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

Device Used : cuda:0


# Pytorch utils

In [41]:
def evaluate_model(model, valid_dataset, valid_dataset_len):
    since = time.time()

    # Each epoch has a training and validation phase
    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0
    y = torch.Tensor()
    pred_y = torch.Tensor()
    criterion = nn.CrossEntropyLoss()

    # Iterate over data.
    for inputs, labels in tqdm(valid_dataset):
        inputs = torch.cat([inputs,inputs,inputs],1)
        inputs = inputs.to(device).float()
        labels = labels.to(device)
        labels = labels.long()
        y = torch.cat([y, labels.cpu()])
        
        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            probs, preds = torch.max(outputs, 1)
            
            #print(outputs.shape)
            loss = criterion(outputs, labels)

            pred_y = torch.cat([pred_y, outputs.cpu()])

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / valid_dataset_len
    epoch_acc = running_corrects.double() / valid_dataset_len
    AUC_ROC = roc_auc_score(y.detach().numpy(), F.softmax(pred_y, dim=1).detach().numpy(), multi_class="ovr")  

    
    time_elapsed = time.time() - since
    print(f'VALIDATION : Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} AUC ROC : {AUC_ROC:.4f} Time taken : {round(time.time() - since, 2)}')

In [54]:
def training_epoch(model, optimizer, scheduler, 
                   train_dataloader, train_dataset_size,
                   valid_dataloader, valid_dataset_size, 
                   num_epochs=25, update_every=5,
                   clean_every = 10):
    since = time.time()

    returnDict = {"trainingLoss" : [], "trainingAccuracy" : [], "epochs" : [], "trainingAUC" : []}
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        returnDict["epochs"].append(epoch)

        since2 = time.time()
        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0
        y = torch.Tensor()
        pred_y = torch.Tensor()
        criterion = nn.CrossEntropyLoss()

        i = 0
        samples_so_far = 0
        # Iterate over data.
        progress_bar = tqdm(train_dataloader)
        for inputs, labels in progress_bar:
            i += 1
            samples_so_far += inputs.shape[0]
            inputs = torch.cat([inputs,inputs,inputs],1)
            inputs = inputs.to(device).float()
            labels = labels.to(device)
            labels = labels.long()

            y = torch.cat([y, labels.cpu()])
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs).softmax(dim=1)
                probs, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                pred_y = torch.cat([pred_y, outputs.cpu()])

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            if i % update_every == 0:
                rl_avg = running_loss / samples_so_far
                acc_avg = running_corrects / samples_so_far
                AUC_ROC = roc_auc_score(y.detach().numpy(), F.softmax(pred_y, dim=1).detach().numpy(), multi_class="ovr")  

                progress_bar.set_description(f"loss = {rl_avg}, accuracy : {acc_avg}, AUC ROC {AUC_ROC}")
            
            if clean_every != -1 and i % clean_every == 0:
                gc.collect()
            scheduler.step()
            
        epoch_loss = running_loss / train_dataset_size
        epoch_acc = running_corrects.double() / train_dataset_size
        AUC_ROC = roc_auc_score(y.detach().numpy(), F.softmax(pred_y, dim=1).detach().numpy(), multi_class="ovr")  

        returnDict["trainingLoss"].append(epoch_loss)
        returnDict["trainingAccuracy"].append(epoch_acc.cpu().item())
        returnDict["trainingAUC"].append(AUC_ROC)

        print(f'TRAINING ; Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} AUC ROC : {AUC_ROC:.4f} Time taken : {round(time.time() - since2, 2)}')
        evaluate_model(model, valid_dataloader, valid_dataset_size)

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    # load best model weights
    returnDict["best_model"] = model
    return returnDict

In [55]:
def plotResults(results):
    plt.figure(figsize=(16,12))
    plt.subplot(3,1, 1)
    plt.plot(results['epochs'], results['trainingLoss'], label="training Loss")
    plt.legend()

    plt.subplot(3,1, 2)
    plt.plot(results['epochs'], results['trainingAccuracy'], label="training Accuracy")
    plt.legend()
  
  
    plt.subplot(3,1, 3)
    plt.plot(results['epochs'], results['trainingAUC'], label="training ROC")
    plt.legend()

    plt.show()

In [56]:
def doExperiment(model_ft, epochs=10, learning_rate=1e-3, plot=True, _summary=True, input_size=(3,32,32)):
  global train_dl, train_ds, val_dl, val_ds
  if _summary:
    print(summary(model_ft, input_size=input_size, device="cpu"))
  
  model_ft = model_ft.to(device)
  optim = pt_optim.Adam(model_ft.parameters(), lr=learning_rate, weight_decay=3e-5)
  lr_sched = pt_lr_scheduler.CyclicLR(optim, learning_rate, 5 * learning_rate, step_size_up=round(len(train_ds) / batch_size), cycle_momentum=False)

  dats = training_epoch(model_ft, optim, lr_sched, 
                        train_dl,  len(train_ds) , 
                        val_dl, len(val_ds),
                        num_epochs=epochs)
  
  if plot:
    plotResults(dats)
  return dats

# Train RESNET

In [ ]:
model = resnet18(True)
model.fc = torch.nn.Linear(512,3)
doExperiment(model, learning_rate=3e-4, input_size=(3, 125, 125))

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 63, 63]           9,408
       BatchNorm2d-2           [-1, 64, 63, 63]             128
              ReLU-3           [-1, 64, 63, 63]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
       BasicBlock-11           [-1, 64, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          36,864
      BatchNorm2d-13           [-1, 64, 32, 32]             128
             ReLU-14           [-1, 64,

loss = 1.1327543258666992, accuracy : 0.32901784777641296, AUC ROC 0.49417781018402734:   8%|▊         | 36/469 [00:09<01:48,  3.98it/s]

In [15]:
HUB_URL = "SharanSMenon/swin-transformer-hub:main"
MODEL_NAME = "swin_tiny_patch4_window7_224"
## check hubconf for more models.
#model = torch.hub.load(HUB_URL, MODEL_NAME, pretrained=True).cuda()
#model.head = torch.nn.Linear(768,3)
#model = model.cuda()


model = resnet18(True)
model.fc = torch.nn.Linear(512,3)



loss_fn = torch.nn.CrossEntropyLoss()


N_EPOCHS = 50
import matplotlib.pyplot as plt

for i in range(N_EPOCHS):
    accuracy = 0
    total_loss = 0
    for x, y in tqdm(train_dl):
        
        x = torch.cat([x,x,x],1).float()
        x = x.cuda()
        y_one_hot = torch.nn.functional.one_hot(y, num_classes=3).float()
        y_one_hot = y_one_hot.cuda()

        output = model(x).softmax(dim=1)
        #print(output)
        loss = loss_fn(output, y_one_hot)
        label = torch.argmax(output,1).cpu()
        
        
        accuracy += torch.sum(label == y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss
    
    test_accuracy = 0    
    with torch.no_grad():
    
        for x, y in test_dl:
            
            x = torch.cat([x,x,x],1).float()
            x = x.cuda()
            y_one_hot = torch.nn.functional.one_hot(y, num_classes=3).float()
            y_one_hot = y_one_hot.cuda()

            output = model(x)
            loss = loss_fn(output, y_one_hot)
            label = torch.argmax(output,1).cpu()
            test_accuracy += torch.sum(label == y)
            
    print(f"train_acc : {accuracy / len(train_ds)}, test_acc : {test_accuracy / len(test_ds)}")

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Tesla T4


100%|██████████| 469/469 [01:34<00:00,  4.97it/s]


train_acc : 0.33623334765434265, test_acc : 0.33506667613983154


100%|██████████| 469/469 [01:34<00:00,  4.96it/s]


...

train_acc : 0.8955000042915344, test_acc : 0.878933310508728


100%|██████████| 469/469 [01:36<00:00,  4.88it/s]


train_acc : 0.9022666811943054, test_acc : 0.8848000168800354


  5%|▌         | 25/469 [00:04<01:19,  5.58it/s]


KeyboardInterrupt: ignored

[]

In [22]:
torch.save(model, os.path.join(SAVE_MODEL_PATH, "Resnet18.pth"))